In [18]:
#!pip install pdoc weasyprint
#!pip install markdown
#!pip install pdfkit
#https://wkhtmltopdf.org/downloads.html

In [26]:
import ast
import os
import markdown
import pdfkit

# Define folder and file paths
FOLDER_DOC = "docs"
os.makedirs(FOLDER_DOC, exist_ok=True)
module_path = 'AIzymes_013.py'
markdown_path = f'{FOLDER_DOC}/AIzymes_013_docs.md'
html_path = f'{FOLDER_DOC}/AIzymes_013_docs.html'
pdf_path = f'{FOLDER_DOC}/AIzymes_013_docs.pdf'

# Configure pdfkit to use wkhtmltopdf
config = pdfkit.configuration(wkhtmltopdf=r"C:\Program Files\wkhtmltopdf\bin\wkhtmltopdf.exe")

# Step 1: Extract docstrings and save as Markdown
def extract_docstrings(filepath):
    with open(filepath, "r") as file:
        tree = ast.parse(file.read())

    docstrings = []
    for node in ast.walk(tree):
        if isinstance(node, (ast.FunctionDef, ast.ClassDef, ast.Module)):
            docstring = ast.get_docstring(node)
            if docstring:
                if isinstance(node, ast.Module):
                    header = "# Module: AIzymes_013\n\n"
                elif isinstance(node, ast.ClassDef):
                    header = f"## Class: {node.name}\n\n"
                elif isinstance(node, ast.FunctionDef):
                    header = f"### Function: {node.name}\n\n"
                
                # Improved formatting for parameter section
                formatted_docstring = []
                for line in docstring.splitlines():
                    line = line.strip()
                    # Add extra line breaks for each parameter and a tab before descriptions
                    if "Parameters:" in line:
                        formatted_docstring.append("**Parameters:**\n")
                    elif ": " in line:
                        parts = line.split(": ", 1)
                        formatted_docstring.append(f"**{parts[0].strip()}**:\t{parts[1].strip()}\n")
                    else:
                        formatted_docstring.append(line)

                # Join with double newlines to add space between parameters
                docstrings.append(header + "\n".join(formatted_docstring) + "\n\n")
    return "\n".join(docstrings)

docstring_content = extract_docstrings(module_path)
with open(markdown_path, "w") as file:
    file.write(docstring_content)

# Step 2: Convert Markdown to HTML
with open(markdown_path, "r") as f:
    text = f.read()
    html_content = markdown.markdown(text)

# Optional: Add CSS for styling
css = """
<style>
    body { font-family: Arial, sans-serif; margin: 20px; }
    h1 { color: #4CAF50; font-size: 2em; }
    h2 { color: #333; font-size: 1.5em; }
    h3 { color: #555; font-size: 1.2em; }
    p { font-size: 1em; line-height: 1.6; }
</style>
"""
html_content = css + html_content

with open(html_path, "w") as f:
    f.write(html_content)

# Step 3: Convert HTML to PDF using pdfkit
pdfkit.from_file(html_path, pdf_path, configuration=config, options={
    'quiet': '',
    'dpi': 300,
    'disable-smart-shrinking': '',
    'enable-local-file-access': ''
})
print(f"PDF generated at {pdf_path}")



PDF generated at docs/AIzymes_013_docs.pdf


In [27]:
!ls

AIzymes_013.py
FieldTools.py
OLD
README.md
__pycache__
cst_weight and kbt_weight optimization.ipynb
design_ESMfold_001.py
design_LigandMPNN_001.py
design_ProteinMPNN_001.py
design_RosettaDesign_001.py
design_RosettaRelax_001.py
design_match_001.py
docs
generate_documentation.ipynb
helper_001.py
main_design_001.py
main_running_001.py
main_scripts_001.py
main_startup_001.py
plotting_001 Backup.py
plotting_001.py
scoring_efields_001.py
setup_system_001.py
